# Working with AutoGluon

This guide goes through how to use this package with autogluon hyperparameter tuning package.


## Load the train and test datasets

First, we'll go ahead and grab the train and test sets for the [arcene](https://archive.ics.uci.edu/ml/datasets/Arcene) data set using the `tabben` package.

In [1]:
from tabben.datasets import OpenTabularDataset

train_ds = OpenTabularDataset('./data/', 'arcene')
test_ds = OpenTabularDataset('./data/', 'arcene', split='test')

Data already available at `data/arcene.npz`
Data already available at `data/arcene.npz`


This dataset has a large number of features, some of which are intentionally meaningless. (The attributes are not assigned to meaningful concepts either.)

In [2]:
print(f'Number of Attributes: {train_ds.num_inputs}')
print(f'Attributes: {train_ds.input_attributes}')

Number of Attributes: 10000
Attributes: ['0' '1' '2' ... '9997' '9998' '9999']


For this dataset, we can get the metric functions that we should use (for consistency across everyone's runs) for evaluating on the test set. Autogluon will only use 1 metric (that it tests on its validation data set), so we just choose one of them.

In [3]:
from tabben.autogluon import get_metrics

eval_metrics = get_metrics(train_ds.task, classes=train_ds.num_classes)

print(eval_metrics)

[binary auroc, ap score, mcc]


## Train the set of models

Now we can use [autogluon](https://auto.gluon.ai/stable/index.html) to automatically train a large set of different models and evaluate on all of them. We'll use the `TabularPredictor` class from autogluon.

In [4]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(
    eval_metric=eval_metrics[0],
    label=train_ds.output_attributes[0], 
    path='ag-covertype')

In [5]:
predictor.fit(
    train_ds.dataframe().head(300),  # artificially reduce the size of the dataset for faster demo
    presets='medium_quality_faster_train')

Presets specified: ['medium_quality_faster_train']
Beginning AutoGluon training ...
AutoGluon will save models to "ag-covertype/"
AutoGluon Version:  0.3.1
Train Data Rows:    100
Train Data Columns: 10000
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, -1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = -1
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (1) vs negative (-1) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6

Metric binary auroc is not supported by this model - using log_loss instead
No improvement since epoch 2: early stopping
	0.8636	 = Validation score   (binary auroc)
	25.08s	 = Training   runtime
	3.78s	 = Validation runtime
Fitting model: XGBoost ...
		tuple index out of range
Detailed Traceback:
Traceback (most recent call last):
  File "_catboost.pyx", line 1249, in _catboost._MetricEval
  File "/home/tmthy/.local/lib/python3.8/site-packages/autogluon/tabular/models/catboost/catboost_utils.py", line 44, in evaluate
    raise NotImplementedError('Custom Catboost Binary prob metrics are not supported by AutoGluon.')
NotImplementedError: Custom Catboost Binary prob metrics are not supported by AutoGluon.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tmthy/.local/lib/python3.8/site-packages/autogluon/tabular/trainer/abstract_trainer.py", line 962, in _train_and_save
    model = self._train_single(X, y, model, X_val

[18:45:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Fitting model: NeuralNetMXNet ...
		Unable to import dependency mxnet. A quick tip is to install via `pip install mxnet --upgrade`, or `pip install mxnet_cu101 --upgrade`
Fitting model: LightGBMLarge ...
/home/tmthy/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
	0.8535	 = Validation score   (binary auroc)
	8.76s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.9545	 = Validation score   (binary auroc)
	0.62s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 163.87s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("ag-covertype/")


We can check to make sure that autogluon inferred the correct task (binary classification for this dataset).

In [6]:
print(predictor.problem_type)
print(predictor.feature_metadata)

binary
('int', [])       : 9824 | ['0', '1', '2', '3', '4', ...]
('int', ['bool']) :   96 | ['277', '471', '519', '549', '652', ...]


## Evaluate the model

Now, we're ready to evaluate our dataset. We can evaluate using autogluon's `leaderboard` method and supply our extra metrics that we want to compare by.

In [7]:
X_test = test_ds.dataframe().drop(columns=test_ds.output_attributes)

y_pred = predictor.predict(X_test)

In [8]:
predictor.leaderboard(test_ds.dataframe(), silent=True, extra_metrics=eval_metrics[1:])

,model,score_test,ap score,mcc,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist,0.844156,0.769047,0.694684,0.787879,0.076603,0.022398,5.369502,0.076603,0.022398,5.369502,1,True,2
1,KNeighborsUnif,0.844156,0.769047,0.694684,0.787879,0.085372,0.037480,5.608608,0.085372,0.037480,5.608608,1,True,1
2,RandomForestGini,0.803571,0.713659,0.612764,0.787879,0.092127,0.053324,9.698220,0.092127,0.053324,9.698220,1,True,5
3,WeightedEnsemble_L2,0.795455,0.680071,0.586756,0.954545,8.265040,3.945264,50.675081,0.006439,0.000839,0.621877,2,True,11
4,RandomForestEntr,0.794643,0.699286,0.592663,0.944444,0.095222,0.052226,9.232696,0.095222,0.052226,9.232696,1,True,6
5,NeuralNetFastAI,0.780844,0.646420,0.580864,0.863636,7.941949,3.782897,25.079172,7.941949,3.782897,25.079172,1,True,9
6,ExtraTreesGini,0.758117,0.666585,0.530757,0.898990,0.091165,0.067327,9.804221,0.091165,0.067327,9.804221,1,True,7
7,ExtraTreesEntr,0.758117,0.666585,0.530757,0.898990,0.108916,0.050196,9.938987,0.108916,0.050196,9.938987,1,True,8
8,LightGBMXT,0.742695,0.638891,0.489884,0.898990,0.075289,0.036294,7.208091,0.075289,0.036294,7.208091,1,True,3
9,LightGBM,0.625000,0.525556,0.258535,0.909091,0.144242,0.033991,6.979780,0.144242,0.033991,6.979780,1,True,4


(If you're looking at the leaderboard in the notebook, the 'score_test' column represents the auroc metric that was passed to the `TabularPredictor` constructor.)

---

This code was last run with the following versions (if you're looking at the no-output webpage, see the notebook in the repository for versions):

In [9]:
from importlib.metadata import version

packages = ['autogluon', 'tabben']

for pkg in packages:
    print(f'{pkg}: {version(pkg)}')

autogluon: 0.3.1
tabben: 0.0.3.dev0
